In [1]:
import pandas as pd
import numpy as np
import re
import os

from tqdm import tqdm
tqdm.pandas()

/home/barry/miniconda3/envs/bertenv2/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# 1 匯入資料

## 1-1 匯入危機公司新聞

### 1-1-1 匯入TEJ危機公司新聞

In [2]:
tej_distress = pd.read_csv('data/2018_2017_tej_distress_0423.csv')
print(tej_distress.shape)
tej_distress.head(2)

(3444, 5)


,公司碼,年月,則次,序號,新聞內容
0,2474 可成,2017/12/29,01,10,可成旗下子公司可成科技( 宿遷) 及可功科技( 宿遷)28 日分別斥資6.2 億元及6.3 ...
1,1611 中電,2017/12/28,01,10,公開資訊觀測站重大訊息公告 ...


### 1-1-2 匯入UDN危機公司新聞

In [3]:
# 設定路徑，將該資料夾內所有檔案名稱列出來
Folder_Path = 'data/udn_distress/'
file_list = os.listdir(Folder_Path)
# file_list

In [4]:
udn_distress = pd.DataFrame(columns=['company', 'title', 'time', 'content'])

for file in file_list:
    df_temp = pd.read_csv('data/udn_distress/' + file)
    udn_distress = pd.concat([udn_distress, df_temp])
udn_distress = udn_distress.drop('title', axis=1).dropna()
print(udn_distress.shape)
udn_distress.head(2)

(2589, 3)


,company,time,content
0,2331 精英,【2009-04-13/經濟日報/C3版/市場焦點】,（\r\n）積極搶攻電信業自有品牌小筆電市場宣告開花結果。\r\n主管表示，\r\n近期拿下...
1,2331 精英,【2009-05-08/經濟日報/C3版/市場焦點】,（\r\n）營運出現大轉機，新任總經理許志宏昨（7）日上任，\r\n將再搶下大陸第二階段家電...


### 1-1-3 （針對UDN）擷取日期，把日期格式轉換為yyyy/mm/dd

In [5]:
from datetime import date
import datetime

#測試一下

s = udn_distress['time'].iloc[4]
print(s)
match = re.search('\d+-\d+-\d+', s)
date = str(match.group(0))
date = date.replace('-', '/')
print(date)
date = datetime.datetime.strptime(date, '%Y/%m/%d').date()
# date = datetime.datetime.strptime(match.group(0), '%Y-%m-%d').date()
print(date)

【2009-05-25/經濟日報/C3版/市場焦點】
2009/05/25
2009-05-25


In [6]:
dates = []
for time in udn_distress['time']:
    s = time
    match = re.search('\d+-\d+-\d+', s)
    date = str(match.group(0))
    date = date.replace('-', '/')
    dates.append(date)
udn_distress['date'] = dates
# udn_distress.drop('time', axis=1, inplace=True)
print(udn_distress.shape)
udn_distress.head(2)

(2589, 4)


,company,time,content,date
0,2331 精英,【2009-04-13/經濟日報/C3版/市場焦點】,（\r\n）積極搶攻電信業自有品牌小筆電市場宣告開花結果。\r\n主管表示，\r\n近期拿下...,2009/04/13
1,2331 精英,【2009-05-08/經濟日報/C3版/市場焦點】,（\r\n）營運出現大轉機，新任總經理許志宏昨（7）日上任，\r\n將再搶下大陸第二階段家電...,2009/05/08


In [7]:
# 把一些字串刪掉改空格
udn_distress['content'] = udn_distress['content'].astype(str).str.replace('\r' , ' ').str.replace('\t' , ' ').str.replace("\n", " ").str.replace(",", " ").str.replace('"' , ' ').str.replace("'" , ' ')
udn_distress['company'] = udn_distress['company'].astype(str).str.replace('\r' , ' ').str.replace('\t' , ' ').str.replace("\n", " ").str.replace(",", " ").str.replace('"' , ' ').str.replace("'" , ' ')
print(udn_distress.shape)
udn_distress.head(2)

(2589, 4)


,company,time,content,date
0,2331 精英,【2009-04-13/經濟日報/C3版/市場焦點】,（ ）積極搶攻電信業自有品牌小筆電市場宣告開花結果。 主管表示， 近期拿下遠傳（490...,2009/04/13
1,2331 精英,【2009-05-08/經濟日報/C3版/市場焦點】,（ ）營運出現大轉機，新任總經理許志宏昨（7）日上任， 將再搶下大陸第二階段家電下鄉大單...,2009/05/08


In [8]:
udn_distress = udn_distress.loc[:, ['company', 'content', 'date']]
print(udn_distress.shape)
udn_distress.head(2)

(2589, 3)


,company,content,date
0,2331 精英,（ ）積極搶攻電信業自有品牌小筆電市場宣告開花結果。 主管表示， 近期拿下遠傳（490...,2009/04/13
1,2331 精英,（ ）營運出現大轉機，新任總經理許志宏昨（7）日上任， 將再搶下大陸第二階段家電下鄉大單...,2009/05/08


In [ ]:
udn_distress.to_csv("data/udn_distress_news.csv", index=False)

In [9]:
udn_distress = pd.read_csv("data/udn_distress_news.csv", encoding="utf-8")
print(udn_distress.shape)
udn_distress.head(2)

(2589, 3)


,company,content,date
0,2331 精英,（ ）積極搶攻電信業自有品牌小筆電市場宣告開花結果。 主管表示， 近期拿下遠傳（490...,2009/04/13
1,2331 精英,（ ）營運出現大轉機，新任總經理許志宏昨（7）日上任， 將再搶下大陸第二階段家電下鄉大單...,2009/05/08


### 1-1-4 合併危機公司資料集為 distress_news

In [10]:
distress_news = pd.DataFrame(columns=['company', 'date', 'content'])
company = udn_distress['company']
company = company.append(tej_distress['公司碼'])
date = udn_distress['date']
date = date.append(tej_distress['年月'])
content = udn_distress['content']
content = content.append(tej_distress['新聞內容'])
distress_news['company'] = company
distress_news['date'] = date
distress_news['content'] = content
print(distress_news.shape)
distress_news.head(2)

(6033, 3)


,company,date,content
0,2331 精英,2009/04/13,（ ）積極搶攻電信業自有品牌小筆電市場宣告開花結果。 主管表示， 近期拿下遠傳（490...
1,2331 精英,2009/05/08,（ ）營運出現大轉機，新任總經理許志宏昨（7）日上任， 將再搶下大陸第二階段家電下鄉大單...


In [ ]:
distress_news.to_csv("data/tej_udn_distress_news.csv", index=False)

In [11]:
distress_news = pd.read_csv("data/tej_udn_distress_news.csv", encoding="utf-8")
print(distress_news.shape)
distress_news.head(2)

(6033, 3)


,company,date,content
0,2331 精英,2009/04/13,（ ）積極搶攻電信業自有品牌小筆電市場宣告開花結果。 主管表示， 近期拿下遠傳（490...
1,2331 精英,2009/05/08,（ ）營運出現大轉機，新任總經理許志宏昨（7）日上任， 將再搶下大陸第二階段家電下鄉大單...


## 1-2 匯入正常公司新聞

### 1-2-1 匯入TFJ正常公司新聞

In [12]:
tej_normal = pd.read_csv('data/2018_2017_tej_normal_0423.csv')
print(tej_normal.shape)
tej_normal.head(2)

(7269, 5)


,公司碼,年月,則次,序號,新聞內容
0,9946 三發地產,2017/12/26,01,10,三發地產(9946)昨(25)日舉行法說會時表示，總銷50億元的高雄「三發晶沙」、總銷50億...
1,2049 上銀,2017/12/11,01,10,上銀(2049)公佈11月合併營收首度突破20億元，達20.41億元，月增3.5%、年增34...


### 1-2-2 匯入UDN正常公司新聞（0422：正常公司的UDN資料蒐集不完整，暫不採納）

In [ ]:
# Folder_Path = '/home/michael/Documents/M10609101_Thesis/News/udn/normal/'
# file_list = os.listdir(Folder_Path)
# # file_list

In [ ]:
# udn_normal = pd.DataFrame(columns=['company', 'title', 'time', 'content'])

# for file in file_list:
#     df_temp = pd.read_csv('News/udn/normal/' + file)
#     udn_normal = pd.concat([udn_normal, df_temp])
# udn_normal = udn_normal.drop('title', axis=1).dropna()
# udn_normal.head()

In [ ]:
# len(udn_normal)

### 1-2-3 （針對UDN）擷取日期，把日期格式轉換為yyyy/mm/dd（0422：正常公司的UDN資料蒐集不完整，暫不採納）

In [ ]:
# dates = []
# for time in udn_normal['time']:
#     s = time
#     match = re.search('\d+-\d+-\d+', s)
#     date = str(match.group(0))
#     date = date.replace('-', '/')
#     dates.append(date)
# udn_normal['date'] = dates
# # udn_normal.drop('time', axis=1, inplace=True)
# udn_normal.tail()

### 1-2-4 合併危機公司資料集為 normal_news

In [13]:
normal_news = pd.DataFrame(columns=['company', 'date', 'content'])
normal_news['company'] = tej_normal['公司碼']
normal_news['date'] = tej_normal['年月']
normal_news['content'] = tej_normal['新聞內容']
print(normal_news.shape)
normal_news.head(2)

(7269, 3)


,company,date,content
0,9946 三發地產,2017/12/26,三發地產(9946)昨(25)日舉行法說會時表示，總銷50億元的高雄「三發晶沙」、總銷50億...
1,2049 上銀,2017/12/11,上銀(2049)公佈11月合併營收首度突破20億元，達20.41億元，月增3.5%、年增34...


In [ ]:
# normal_news = pd.DataFrame(columns=['company', 'date', 'content'])
# company = udn_normal['company']
# company = company.append(tej_normal['公司碼'])
# date = udn_normal['date']
# date = date.append(tej_normal['年月'])
# content = udn_normal['content']
# content = content.append(tej_normal['新聞內容'])
# normal_news['company'] = company
# normal_news['date'] = date
# normal_news['content'] = content
# normal_news.head()

In [ ]:
normal_news.to_csv("data/tej_normal_news.csv", index=False)

In [14]:
normal_news = pd.read_csv("data/tej_normal_news.csv", encoding="utf-8")
print(normal_news.shape)
normal_news.head(2)

(7269, 3)


,company,date,content
0,9946 三發地產,2017/12/26,三發地產(9946)昨(25)日舉行法說會時表示，總銷50億元的高雄「三發晶沙」、總銷50億...
1,2049 上銀,2017/12/11,上銀(2049)公佈11月合併營收首度突破20億元，達20.41億元，月增3.5%、年增34...


In [15]:
all_news = pd.concat([normal_news, distress_news])   # 上下合併
print(all_news.shape)
all_news.head(2)

(13302, 3)


,company,date,content
0,9946 三發地產,2017/12/26,三發地產(9946)昨(25)日舉行法說會時表示，總銷50億元的高雄「三發晶沙」、總銷50億...
1,2049 上銀,2017/12/11,上銀(2049)公佈11月合併營收首度突破20億元，達20.41億元，月增3.5%、年增34...


In [16]:
all_news['date'] = pd.to_datetime(all_news['date'])
all_news = all_news.sort_values(by=["date","company"]).reset_index(drop=True) # 用post_time排序 # 在重設index
print(all_news.shape)
all_news.head(2)

(13302, 3)


,company,date,content
0,2316 楠梓電,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...
1,5469 瀚宇博,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...


In [5]:
all_news.to_csv("data/tej_udn_all_news.csv", index=False)

## 2-3 每間公司風險分布
使用意藍2015年風險資料集、經過BERT主題模型產生的model 將每篇文章輸出1*5的output

13302篇文章就有13302*5

In [1]:
import pandas as pd
import numpy as np
import re
import os

from tqdm import tqdm
tqdm.pandas()

import tensorflow as tf
from transformers import BertTokenizer, TFBertModel, TFBertForSequenceClassification, AdamW

from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import Dense, Embedding, SpatialDropout1D, Dropout, Activation, Flatten, InputLayer

/home/barry/miniconda3/envs/bertenv/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [13]:
all_news = pd.read_csv("data/tej_udn_all_news.csv", encoding="utf-8")
all_news = all_news.sort_values(by=["date","company"]).reset_index(drop=True)
print(all_news.shape)
all_news.head(2)

(13302, 3)


,company,date,content
0,2316 楠梓電,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...
1,2316 楠梓電,2007-01-01,台灣表面黏著科技（6278）受惠台灣近年面板蓬勃發展，去年成為最大的印刷電路板表面黏著科技廠...


In [3]:
all_news.describe()

,company,date,content
count,13302,13302,13302
unique,240,3038,12746
top,2330 台積電,2008-01-17,中華信用評等公司昨日確認台塑集團四家核心公司，包括台塑、南亞塑膠、台化、台塑石化，長期企業信...
freq,613,20,4


In [4]:
all_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13302 entries, 0 to 13301
Data columns (total 3 columns):
company    13302 non-null object
date       13302 non-null object
content    13302 non-null object
dtypes: object(3)
memory usage: 311.9+ KB


In [5]:
# 文章字數 > 510了話 去尾
# 字數小於512-2 因為還有CLS SEP
def remove_510(text):
    if len(text) > 510:
        text = text[:510]    # 只取前510個字
    return text

In [6]:
all_news["content"] = all_news["content"].progress_apply(remove_510)
print(all_news.shape)
all_news.head(2)

100%|██████████| 13302/13302 [00:00<00:00, 933190.02it/s]

(13302, 3)


,company,date,content
0,2316 楠梓電,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...
1,5469 瀚宇博,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...


In [7]:
# Load pre-trained model tokenizer, to convert our text into tokens that correspond to BERT’s vocabulary.
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

def input_ids_all(text):
#    tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
    tokens = tokenizer.tokenize(text)        # 每個字切詞成一個list
    input_ids = tokenizer.convert_tokens_to_ids(tokens)  # 每個字轉成id
    input_ids = tokenizer.build_inputs_with_special_tokens(input_ids)    # 句子前後加上 CLS SEP 的 id
    input_ids = np.array(input_ids)          # list 轉 numpy
    if len(input_ids) < 512:
        n = 512 - len(input_ids)
        input_ids = np.pad(input_ids, (0, n), mode ='constant', constant_values=(0))  
        # array右邊append n 個 0  補長度到512
    return input_ids

In [8]:
# df['text2'] = df['text']].apply(lambda x: " ".join(jieba.cut(x)))
# df_news['tokens'] = df_news['text'].apply(lambda x:  tokenizer.tokenize(x) )
all_news['input_ids'] = all_news['content'].progress_apply(input_ids_all)
print(all_news.shape)
all_news.head(2)

100%|██████████| 13302/13302 [00:17<00:00, 770.21it/s]

(13302, 4)


,company,date,content,input_ids
0,2316 楠梓電,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...,"[101, 1313, 1170, 7442, 6662, 3352, 4496, 3511..."
1,5469 瀚宇博,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...,"[101, 1313, 1170, 7442, 6662, 3352, 4496, 3511..."


In [12]:
# df_news['input_ids'].to_numpy() 出來不是一個2d numpy 
# 只好用for loop一個一個拿出來合併
# 用np vstack超級慢 不知道為何  改用最外層是list append
input_ids = []      # list
for index, row in all_news.iterrows():  
    np_1d = all_news.loc[index,'input_ids']    # 1d np arrary
    input_ids.append(np_1d)                       # 1d np的 list # list[np_1, np_2, np_3, ....]


input_ids = np.array(input_ids)                       # 轉成2d np
print(input_ids.shape)
input_ids

(13302, 512)


array([[ 101, 1313, 1170, ...,    0,    0,    0],
       [ 101, 1313, 1170, ...,    0,    0,    0],
       [ 101, 1313, 1170, ...,    0,    0,    0],
       ...,
       [ 101, 1377, 2768, ...,    0,    0,    0],
       [ 101, 8020, 8021, ...,    0,    0,    0],
       [ 101, 3189, 3315, ...,    0,    0,    0]])

In [13]:
# Recreate the exact same model, including its weights and the optimizer
# 讀取存的model
input_layer = Input(shape = (512,), dtype='int64')  # 預設是float 要改成input_id的int64
load_model = tf.keras.models.load_model('model/model_bert_eland_softmax_1')(input_layer)
model_1 = Model(inputs=input_layer, outputs=load_model)

# Show the model architecture
model_1.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512)]             0         
_________________________________________________________________
model (Model)                (None, 5)                 104233733 
Total params: 104,233,733
Trainable params: 104,233,733
Non-trainable params: 0
_________________________________________________________________


In [14]:
%%time
predictions = model_1.predict(input_ids)   # 輸出的是n*5的編碼值array
print(predictions.shape)
predictions

(13302, 5)
CPU times: user 1min 32s, sys: 22.6 s, total: 1min 54s
Wall time: 4min 43s


array([[9.9999940e-01, 3.0025442e-15, 5.4861158e-07, 7.8284220e-16,
        8.5143589e-14],
       [9.9999952e-01, 1.7350038e-15, 4.4569668e-07, 1.8591876e-15,
        1.1685662e-13],
       [9.9999952e-01, 2.0303369e-15, 4.5428749e-07, 4.6447030e-16,
        2.1901956e-14],
       ...,
       [1.0000000e+00, 1.8669188e-32, 8.6114038e-23, 3.9657870e-38,
        6.4282299e-29],
       [1.0000000e+00, 2.0209136e-20, 2.5738619e-12, 3.7298192e-24,
        5.7666145e-14],
       [8.9137890e-22, 2.7357174e-22, 6.2450786e-17, 5.4473147e-28,
        1.0000000e+00]], dtype=float32)

In [15]:
df_predictions = pd.DataFrame(predictions, columns = ["c_0", "c_1", "c_2", "c_3", "c_4"])
print(df_predictions.shape)
df_predictions.head(2)

(13302, 5)


,c_0,c_1,c_2,c_3,c_4
0,0.999999,3.002544e-15,5.486116e-07,7.828422e-16,8.514359e-14
1,1.000000,1.735004e-15,4.456967e-07,1.859188e-15,1.168566e-13


In [17]:
all_news = pd.concat([all_news, df_predictions], axis=1)   # axis=1左右合併 axis=0 上下合併
print(all_news.shape)
all_news.head(2)

(13302, 9)


,company,date,content,input_ids,c_0,c_1,c_2,c_3,c_4
0,2316 楠梓電,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...,"[101, 1313, 1170, 7442, 6662, 3352, 4496, 3511...",0.999999,3.002544e-15,5.486116e-07,7.828422e-16,8.514359e-14
1,5469 瀚宇博,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...,"[101, 1313, 1170, 7442, 6662, 3352, 4496, 3511...",1.000000,1.735004e-15,4.456967e-07,1.859188e-15,1.168566e-13


In [18]:
all_news["c_0"].sum()

9402.648

### 需要宣告一個240公司*10年的dataframe

In [14]:
company_list = all_news.sort_values(by=["company","date"])["company"].unique()  # 把欄位unique的值都挑出來變成numpy
print(len(company_list))
company_list

240


array(['1101 台泥', '1201 味全', '1210 大成', '1215 卜蜂', '1216 統一', '1218 泰山',
       '1220 台榮', '1227 佳格', '1232 大統益', '1233 天仁', '1236 宏亞', '1301 台塑',
       '1307 三芳', '1309 台達化', '1310 台苯', '1314 中石化', '1316 上曜', '1321 大洋',
       '1323 永裕', '1326 台化', '1413 宏洲', '1414 東和', '1432 大魯閣', '1442 名軒',
       '1447 力鵬', '1449 佳和', '1451 年興', '1452 宏益', '1453 大將', '1456 怡華',
       '1457 宜進', '1472 三洋紡', '1475 本盟', '1477 聚陽', '1503 士電', '1506 正道',
       '1507 永大', '1513 中興電', '1515 力山', '1516 川飛', '1521 大億', '1525 江申',
       '1529 樂士', '1532 勤美', '1535 中宇', '1538 正峰新', '1539 巨庭', '1558 伸興',
       '1560 中砂', '1583 程泰', '1611 中電', '1612 宏泰', '1614 三洋電', '1618 合機',
       '1701 中化', '1704 榮化', '1707 葡萄王', '1718 中纖', '1721 三晃', '1723 中碳',
       '1724 台硝', '1726 永記', '1732 毛寶', '1733 五鼎', '1762 中化生', '1786 科妍',
       '1805 寶徠', '1902 台紙', '1903 士紙', '1904 正隆', '1905 華紙', '1907 永豐餘',
       '2002 中鋼', '2008 高興昌', '2013 中鋼構', '2020 美亞', '2025 千興',
       '2027 大成鋼', '2028 威致', '2034 允強', '2049 上銀

In [16]:
temp = []
for company_name in tqdm(company_list):
    for year in range(2008,2018):
        company_name = str(company_name)
        year = int(year)
        temp_np = np.array([company_name , year])    # np(公司名,年分)
        temp.append(temp_np)
        
temp = np.array(temp)
print(len(temp))
temp

100%|██████████| 240/240 [00:00<00:00, 59098.98it/s]

2400


array([['1101 台泥', '2008'],
       ['1101 台泥', '2009'],
       ['1101 台泥', '2010'],
       ...,
       ['9946 三發地產', '2015'],
       ['9946 三發地產', '2016'],
       ['9946 三發地產', '2017']], dtype='<U11')

In [17]:
df_company_year_prob_sum = pd.DataFrame(temp, columns = ['company' , 'date'])  # numpy to pandas 並宣告欄位名
print(df_company_year_prob_sum.shape)
df_company_year_prob_sum

(2400, 2)


,company,date
0,1101 台泥,2008
1,1101 台泥,2009
2,1101 台泥,2010
3,1101 台泥,2011
4,1101 台泥,2012
...,...,...
2395,9946 三發地產,2013
2396,9946 三發地產,2014
2397,9946 三發地產,2015
2398,9946 三發地產,2016


In [18]:
df_company_year_prob_sum["date"] = df_company_year_prob_sum["date"].astype(int)
df_company_year_prob_sum["company"] = df_company_year_prob_sum["company"].astype(str)
df_company_year_prob_sum["c_0"] = 0.       # 建class_0 ~ class_4的欄位
df_company_year_prob_sum["c_1"] = 0.
df_company_year_prob_sum["c_2"] = 0.
df_company_year_prob_sum["c_3"] = 0.
df_company_year_prob_sum["c_4"] = 0.
print(df_company_year_prob_sum.shape)
df_company_year_prob_sum.head(2)

(2400, 7)


,company,date,c_0,c_1,c_2,c_3,c_4
0,1101 台泥,2008,0.0,0.0,0.0,0.0,0.0
1,1101 台泥,2009,0.0,0.0,0.0,0.0,0.0


In [19]:
# pandas 取出 a column contains "A" & b column contains "B" 
# df_company_year_prob_sum[(df_company_year_prob_sum["company"].str.contains("楠梓電")) & (df_company_year_prob_sum["date"].astype(str).str.contains("2009") )]

df_company_year_prob_sum.loc[(df_company_year_prob_sum["company"].str.contains("楠梓電")) 
                    & (df_company_year_prob_sum["date"].astype(str).str.contains("2009") )]

,company,date,c_0,c_1,c_2,c_3,c_4
921,2316 楠梓電,2009,0.0,0.0,0.0,0.0,0.0


### 把每公司 每年 的 class_0相加
### 再丟到df_topic_5_prob

In [123]:
# pandas all_news 取出 company column contains "楠梓電" & date column contains "2008" 
# df_temp = all_news[(all_news["company"].str.contains("楠梓電")) & (all_news["date"].astype(str).str.contains("2008") )]

df_temp = all_news.loc[(all_news["company"].str.contains("楠梓電"))
                    & (all_news["date"].astype(str).str.contains("2008"))]
print(df_temp.shape)
df_temp.head()

(37, 9)


,company,date,content,input_ids,c_0,c_1,c_2,c_3,c_4
1156,2316 楠梓電,2008-01-11,華通（2313）、 （ ）、燿華（2367）及欣興（3037）等四大手機印刷電路板（PC...,"[101, 5836, 6858, 8020, 10602, 8152, 8021, 510...",1.000000e+00,5.157054e-30,5.659684e-22,3.028994e-30,1.360346e-30
1171,2316 楠梓電,2008-01-15,宇環（3276）近年積極籌資，去年下半年，以每股溢價12元募資1.98億元後，昨（14）日董...,"[101, 2126, 4472, 8020, 12094, 8158, 8021, 681...",1.000000e+00,3.418315e-32,3.552168e-24,7.187509e-35,2.747460e-31
1180,2316 楠梓電,2008-01-16,台灣的光電板、手機板大廠楠梓電 (2316)展開新年度人事新布局，由楠梓電集 團執行長陳梅芳...,"[101, 1378, 4124, 4638, 1045, 7442, 3352, 510,...",3.471974e-17,1.122257e-14,3.181559e-08,2.757208e-20,1.000000e+00
1187,2316 楠梓電,2008-01-16,欣興（3037）昨（15）日公布去年兩岸合併營收472.81億元，穩坐台灣印刷電路板（PCB...,"[101, 3615, 5646, 8020, 11068, 8161, 8021, 321...",1.000000e+00,1.507751e-27,1.080081e-19,1.970381e-29,6.713191e-25
1195,2316 楠梓電,2008-01-17,藍營立委選舉大勝帶來台股兩天慶祝行情，但隨昨（16）日重挫，使近來一直「弱不禁風」的上市櫃印...,"[101, 5965, 4245, 4989, 1999, 6908, 5647, 1920...",1.000000e+00,1.549369e-29,4.016741e-18,6.008773e-30,3.861093e-26


In [124]:
# 將 c_0欄位 加總sum()
df_temp["c_0"].sum()

28.857868

In [128]:
# df_company_year_prob_sum[(df_company_year_prob_sum["company"].str.contains("楠梓電") ) & (df_company_year_prob_sum["date"] == 2008) ]["c_0"] = df_temp["c_0"].sum()
# 填入 company欄位 contains 楠梓電 date欄位 contains 2008 的 c_0欄位
# 沒df.loc 填入表格會失敗

df_company_year_prob_sum.loc[(df_company_year_prob_sum["company"].str.contains("楠梓電") )
                    & (df_company_year_prob_sum["date"].astype(str).str.contains( str(2008) )), "c_0"] = df_temp["c_0"].sum()
df_company_year_prob_sum

,company,date,c_0,c_1,c_2,c_3,c_4
0,2316 楠梓電,2008,28.857868,0.0,0.0,0.0,0.0
1,2316 楠梓電,2009,0.000000,0.0,0.0,0.0,0.0
2,2316 楠梓電,2010,0.000000,0.0,0.0,0.0,0.0
3,2316 楠梓電,2011,0.000000,0.0,0.0,0.0,0.0
4,2316 楠梓電,2012,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
2395,1453 大將,2013,0.000000,0.0,0.0,0.0,0.0
2396,1453 大將,2014,0.000000,0.0,0.0,0.0,0.0
2397,1453 大將,2015,0.000000,0.0,0.0,0.0,0.0
2398,1453 大將,2016,0.000000,0.0,0.0,0.0,0.0


In [129]:
# 上面步驟寫成for loop
# for 240公司名list:
#     for 2008-2017年:

for company_name in tqdm(company_list):
    for year in range(2008,2018):
        company_name = str(company_name)
        
        # pandas all_news 取出 company欄位 contains 公司名 & date欄位 contains 年分 
        df_temp = all_news.loc[(all_news["company"].str.contains(company_name))
                    & (all_news["date"].astype(str).str.contains( str(year) ))]
        # 將 c_0 c_1 c_2 c_3 c_4 欄位 加總sum()
        # df_temp["c_0"].sum(), df_temp["c_1"].sum(), df_temp["c_2"].sum() ,df_temp["c_3"].sum() ,df_temp["c_4"].sum()

        # 將 c_0 ~ c_4 欄位 加總sum()
        # 按照 公司名 年分 填入 df_company_year_prob_sum的 c_0 ~ c_4 欄位表格
        df_company_year_prob_sum.loc[(df_company_year_prob_sum["company"].str.contains(company_name) )
                    & (df_company_year_prob_sum["date"].astype(str).str.contains( str(year) )), "c_0"] = df_temp["c_0"].sum()
        df_company_year_prob_sum.loc[(df_topic_5_prob["company"].str.contains(company_name) )
                    & (df_company_year_prob_sum["date"].astype(str).str.contains( str(year) )), "c_1"] = df_temp["c_1"].sum()
        df_company_year_prob_sum.loc[(df_company_year_prob_sum["company"].str.contains(company_name) )
                    & (df_company_year_prob_sum["date"].astype(str).str.contains( str(year) )), "c_2"] = df_temp["c_2"].sum()
        df_company_year_prob_sum.loc[(df_company_year_prob_sum["company"].str.contains(company_name) )
                    & (df_company_year_prob_sum["date"].astype(str).str.contains( str(year) )), "c_3"] = df_temp["c_3"].sum()
        df_company_year_prob_sum.loc[(df_company_year_prob_sum["company"].str.contains(company_name) )
                    & (df_company_year_prob_sum["date"].astype(str).str.contains( str(year) )), "c_4"] = df_temp["c_4"].sum()

df_company_year_prob_sum

100%|██████████| 240/240 [01:02<00:00,  3.84it/s]


,company,date,c_0,c_1,c_2,c_3,c_4
0,2316 楠梓電,2008,28.857868,1.079127e-05,3.993868e+00,3.000000e+00,1.148253e+00
1,2316 楠梓電,2009,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,2316 楠梓電,2010,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,2316 楠梓電,2011,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,2316 楠梓電,2012,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...
2395,1453 大將,2013,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2396,1453 大將,2014,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2397,1453 大將,2015,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2398,1453 大將,2016,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [25]:
df_company_year_prob_sum.to_csv("data/company_year_prob_sum.csv", index=False)

In [26]:
df_company_year_prob_sum = pd.read_csv("data/company_year_prob_sum.csv", encoding="utf-8")
df_company_year_prob_sum["c_0"] = df_company_year_prob_sum["c_0"].astype(float)  # 讀取後將欄位弄成float
df_company_year_prob_sum["c_1"] = df_company_year_prob_sum["c_1"].astype(float)
df_company_year_prob_sum["c_2"] = df_company_year_prob_sum["c_2"].astype(float)
df_company_year_prob_sum["c_3"] = df_company_year_prob_sum["c_3"].astype(float)
df_company_year_prob_sum["c_4"] = df_company_year_prob_sum["c_4"].astype(float)
print(df_company_year_prob_sum.shape)
df_company_year_prob_sum

(2400, 7)


,company,date,c_0,c_1,c_2,c_3,c_4
0,1101 台泥,2008,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,1101 台泥,2009,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,1101 台泥,2010,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,1101 台泥,2011,42.935883,6.660758e-02,4.996792e+00,1.579325e-10,1.200072e+01
4,1101 台泥,2012,34.983147,1.997005e+00,3.350949e+00,7.349652e-12,3.668897e+00
...,...,...,...,...,...,...,...
2395,9946 三發地產,2013,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2396,9946 三發地產,2014,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2397,9946 三發地產,2015,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2398,9946 三發地產,2016,0.000002,1.470138e-13,3.596562e-07,4.796958e-23,1.999998e+00


In [27]:
df_company_year_prob_sum.sort_values(by=["date","company"])  # 先日期排序 再公司名排序

,company,date,c_0,c_1,c_2,c_3,c_4
0,1101 台泥,2008,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
10,1201 味全,2008,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
20,1210 大成,2008,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
30,1215 卜蜂,2008,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
40,1216 統一,2008,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...
2359,9921 巨大,2017,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2369,9933 中鼎,2017,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2379,9934 成霖,2017,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2389,9943 好樂迪,2017,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [28]:
df_company_year_prob_sum.sort_values(by=["company","date"])  # 先公司名排序 再日期排序

,company,date,c_0,c_1,c_2,c_3,c_4
0,1101 台泥,2008,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,1101 台泥,2009,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,1101 台泥,2010,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,1101 台泥,2011,42.935883,6.660758e-02,4.996792e+00,1.579325e-10,1.200072e+01
4,1101 台泥,2012,34.983147,1.997005e+00,3.350949e+00,7.349652e-12,3.668897e+00
...,...,...,...,...,...,...,...
2395,9946 三發地產,2013,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2396,9946 三發地產,2014,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2397,9946 三發地產,2015,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2398,9946 三發地產,2016,0.000002,1.470138e-13,3.596562e-07,4.796958e-23,1.999998e+00


In [29]:
company_list = df_company_year_prob_sum["company"].unique()  # 把欄位unique的值都挑出來變成numpy
print(len(company_list))
company_list

240


array(['1101 台泥', '1201 味全', '1210 大成', '1215 卜蜂', '1216 統一', '1218 泰山',
       '1220 台榮', '1227 佳格', '1232 大統益', '1233 天仁', '1236 宏亞', '1301 台塑',
       '1307 三芳', '1309 台達化', '1310 台苯', '1314 中石化', '1316 上曜', '1321 大洋',
       '1323 永裕', '1326 台化', '1413 宏洲', '1414 東和', '1432 大魯閣', '1442 名軒',
       '1447 力鵬', '1449 佳和', '1451 年興', '1452 宏益', '1453 大將', '1456 怡華',
       '1457 宜進', '1472 三洋紡', '1475 本盟', '1477 聚陽', '1503 士電', '1506 正道',
       '1507 永大', '1513 中興電', '1515 力山', '1516 川飛', '1521 大億', '1525 江申',
       '1529 樂士', '1532 勤美', '1535 中宇', '1538 正峰新', '1539 巨庭', '1558 伸興',
       '1560 中砂', '1583 程泰', '1611 中電', '1612 宏泰', '1614 三洋電', '1618 合機',
       '1701 中化', '1704 榮化', '1707 葡萄王', '1718 中纖', '1721 三晃', '1723 中碳',
       '1724 台硝', '1726 永記', '1732 毛寶', '1733 五鼎', '1762 中化生', '1786 科妍',
       '1805 寶徠', '1902 台紙', '1903 士紙', '1904 正隆', '1905 華紙', '1907 永豐餘',
       '2002 中鋼', '2008 高興昌', '2013 中鋼構', '2020 美亞', '2025 千興',
       '2027 大成鋼', '2028 威致', '2034 允強', '2049 上銀